In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
from scipy import stats as st

- [ ] Анализ регистраций/выдач/CR% в динамике по неделям/месяцам
- [ ] Сколько дней проходит от регистрации до займа
- [ ] Изучение объема выдач
    - [ ] объем выдач в разрезе времени
    - [ ] объем выдач накопительным итогом
- [ ] Изучение среднего чека
    - [ ] средний чек в разрезе времени
    - [ ] классфикация данных, срезы
    - [ ] поиск зависимых параметров

### Чтение данных и объединение таблиц с данными о регистрациях

In [2]:
# читаем данные
regs1 = pd.read_excel('regs1.xlsx')
regs2 = pd.read_excel('regs2.xlsx')
loans = pd.read_excel('loans.xlsx')

In [3]:
# т.к данные с регистрациями предоставлены в 2ух файлах нам необходимо объеденить их в 1
regs = regs1.append(regs2, ignore_index=True)

### Предобработка данных

#### Регистрации

In [4]:
# посмотрим на общую информацию
regs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481285 entries, 0 to 1481284
Data columns (total 4 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   cid              1481285 non-null  int64         
 1   uid              1481285 non-null  object        
 2   ts               1481285 non-null  datetime64[ns]
 3   is_reg_complete  1481285 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 45.2+ MB


In [5]:
# изучим срез данных
regs.sample(5)

,cid,uid,ts,is_reg_complete
843411,2962913,9489D1F6-57BF-405E-B64B-DD46C6B1DB76,2019-01-28 10:42:48.668,1
1425580,4436146,022BC206-3CCD-47DC-9549-6F16E1ABE51E,2020-06-20 16:51:44.438,0
392800,2872087,E5FB8A5A-3919-40E4-8D6E-34D0B4261925,2019-01-10 16:57:02.668,1
937179,3641233,00153902-64CD-424A-AD39-7C1803F96D9D,2019-06-26 14:46:48.736,0
1325267,4526674,C2434E26-D2CC-4842-8832-A0143DF2937C,2020-08-27 11:44:07.471,0


<b>cid</b> - id клиента<br>
<b>uid</b> - id юзера<br>
<b>ts</b> - время начала регистрации<br>
<b>is_reg_complete</b> - отметка о завершени регистрации<br>

In [6]:
# проверим таблицу на наличие пропусков и дубликатов
display(regs.isna().sum())
display(regs.duplicated().sum())

cid                0
uid                0
ts                 0
is_reg_complete    0
dtype: int64

0

<b>Вывод:</b> все ок, двигаемся дальше

#### Займы

In [7]:
# посмотрим на общую информацию
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397272 entries, 0 to 397271
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   cid       397272 non-null  int64         
 1   uid       397272 non-null  object        
 2   start_ts  381801 non-null  datetime64[ns]
 3   value     397272 non-null  float64       
 4   term      397272 non-null  int64         
 5   status    397272 non-null  object        
 6   end_ts    277092 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(2)
memory usage: 21.2+ MB


In [8]:
# изучим срез данных
loans.sample(5)

,cid,uid,start_ts,value,term,status,end_ts
132785,3783613,BCB4946E-545F-4E5F-99D1-EEA5D9BEDCB5,2019-08-18,5300.0,10,Погашен,2019-09-03 23:59:59.999
343513,4480273,6B83D810-585F-4485-8A08-5E54CDAC4F4B,2020-08-13,10000.0,28,Погашен,2020-09-10 23:59:59.999
85887,3118618,D07BB7FF-8DC9-4C6E-9C43-E4EF9A8845BE,2019-05-28,8320.0,30,Погашен,2019-07-15 23:59:59.999
268142,4321568,0B49D576-2BB8-4F19-A168-0094C40852A8,NaT,4520.0,24,Отменён,NaT
300149,4413034,A99B82F1-DB62-4B3E-9654-C32142B54567,2020-06-05,17825.0,36,Погашен,2020-06-17 23:59:59.999


<b>cid</b> - id клиента<br>
<b>uid</b> - id юзера<br>
<b>start_ts</b> - время получения займа<br>
<b>value</b> - сумма<br>
<b>term</b> - срок<br>
<b>status</b> - статус на текущий момент<br>
<b>end_ts</b> - время веозврата займа

In [9]:
# проверим таблицу на наличие пропусков и дубликатов
display(loans.isna().sum())
display(loans.duplicated().sum())

cid              0
uid              0
start_ts     15471
value            0
term             0
status           0
end_ts      120180
dtype: int64

1743

<b>Вывод:</b> есть пропуски и дубликаты, обработаем их

In [10]:
# посмотим на стутусы
loans[loans['start_ts'].isna()]['status'].value_counts()

Отменён           15447
Не подтверждён       24
Name: status, dtype: int64

<b>Вывод:</b> фактически эти статусы говорят о том, что клиент не получил деньги

In [11]:
# Удалим эти строки
loans.dropna(subset=['start_ts'], inplace=True)

In [12]:
loans.isna().sum()

cid              0
uid              0
start_ts         0
value            0
term             0
status           0
end_ts      104709
dtype: int64

In [13]:
# Посмотрим статусы займов, у которых отсутствуетм время погашения
loans[loans['end_ts'].isna()]['status'].value_counts()

Просрочен              41423
На цессии              39043
Активен                14337
На реструктуризации     8616
Отменён                 1025
Списан                   265
Name: status, dtype: int64

In [14]:
# посмотрим, остались ли дубликаты
loans.duplicated().sum()

0

<b>Вывод:</b>
- дубли теперь отсутствуют
- пропуски в end_ts фактически указывают на то, что заем не был закрыт на момент анализа данных

Переходоим к анализу